In [3]:
import os
import sys
py_files_dir = os.path.abspath("../../../vampires_on_sky_calibration/py_files/programs/py_files/")
sys.path.insert(0, py_files_dir)

import em_gain
import numpy as np
import pandas as pd
import helper_functions as funcs

# Examing 04/28/2022 Data

In [4]:
# Define wavelengths
wavelengths = np.array(["625-50", "675-50", "725-50", "750-50", "775-50"])

# Read in CSV file
df = pd.read_csv("../../data/20220428/20220429_New_Masterdarks_Header_Parameters.csv")

for i, wavelength in enumerate(wavelengths):
    # Filter the DataFrame for the specific wavelength
    df_wavelength = df[(df["U_FILTER"] == wavelength)]
    
    # Further filter for each camera from the already filtered DataFrame
    df_cam1 = df_wavelength[df_wavelength["U_CAMERA"] == 1]
    df_cam2 = df_wavelength[df_wavelength["U_CAMERA"] == 2]

    # Extracting fluxes for each camera & FLC state
    FL1 = df_cam1["DARK_SUBTRACTED_TOTAL_MEDIAN_FLUX_FLC_1"].values
    FL2 = df_cam1["DARK_SUBTRACTED_TOTAL_MEDIAN_FLUX_FLC_2"].values
    FR1 = df_cam2["DARK_SUBTRACTED_TOTAL_MEDIAN_FLUX_FLC_1"].values
    FR2 = df_cam2["DARK_SUBTRACTED_TOTAL_MEDIAN_FLUX_FLC_2"].values

    # Adding together both FLC States
    FL = FL1 + FL2
    FR = FR1 + FR2

    # Assuming em_gain.calculate_em_gain_ratio returns normalized_fluxes and em_gain_ratios
    normalized_fluxes, em_gain_ratios = em_gain.calculate_em_gain_ratio(FL, FR)

    print("Number of Data Points: " + str(len(normalized_fluxes)))
    print(f"Median Normalized Flux for {wavelength}: {np.median(normalized_fluxes)}")
    print(f"Median EM Gain Ratio for {wavelength}: {np.median(em_gain_ratios)}")
    print()


Number of Data Points: 64
Median Normalized Flux for 625-50: -0.37148584094361287
Median EM Gain Ratio for 625-50: 2.182112369394222

Number of Data Points: 64
Median Normalized Flux for 675-50: -0.32687068726950047
Median EM Gain Ratio for 675-50: 1.972271901820175

Number of Data Points: 64
Median Normalized Flux for 725-50: -0.3032787251318735
Median EM Gain Ratio for 725-50: 1.870629599294518

Number of Data Points: 72
Median Normalized Flux for 750-50: -0.24548753479580876
Median EM Gain Ratio for 750-50: 1.650744237286565

Number of Data Points: 64
Median Normalized Flux for 775-50: -0.2948902912160366
Median EM Gain Ratio for 775-50: 1.8369183956949486



# Examine Unpolarized Standards

In [5]:
csv_directory = "../../../data/unpolarized/csv"
csv_files = funcs.load_all_files_from_directory(csv_directory, ".csv")

AttributeError: module 'helper_functions' has no attribute 'load_all_files_from_directory'

# Examine Polarized Standards